In [1]:
import numpy as np
import pandas as pd
import random as rd

In [2]:
data = pd.read_csv("data.csv", sep=',')

In [3]:
data.head()

,US,China
0,1,1
1,0,1
2,0,0
3,1,0
4,1,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
US       30 non-null int64
China    30 non-null int64
dtypes: int64(2)
memory usage: 560.0 bytes


In [5]:
# Inititializations
train_num = 29
test_num = 100
y = 0
n = 1
alpha = 0.5 # Learning Rate
gamma = 0.2 # Discount Rate
yy = 0
ny = 0
yn = 0
nn = 0
w = 0.5 # Weight for payoffs
lamda = 0.01 # Rate for updation of w
qtable = [[0 for x in range(2)] for y in range(2)] # Q-table

In [6]:
# Reward Matrix
def reward(state,action):
    
    if state == y: # Opponent move is y
        if action == y:
            return w
        else:
            return (2-3*w)
    else: # Opponent move is n
        if action == y:
            return (3*w-2)
        else:
            return -w

In [7]:
def opponent_Move():
    
    prob_y = (yy+ny/yy+ny+yn+nn)
    prob_n = (yn+nn/yy+ny+yn+nn)
    
    Est_y = max(w,3*w-2)*prob_y
    Est_n = max(2-3*w,-w)*prob_n
    
    if Est_y-Est_n > 0.1:
        return n;
    elif Est_n-Est_y > 0.1:
        return y;
    
    if Est_y>Est_n:
        return y
    elif Est_n>Est_y:
        return n
    else:
        return rd.randint(0,1)

# Train

In [8]:
temp1 = 1
tempyn = yn
tempny = ny

for i in range(train_num):
    
    state = data['China'][i]
    nextstate = data['China'][i+1]
    action = data['US'][i]
    
    r = reward(state,action)
    
    qtable[state][action] = qtable[state][action] + \
    alpha*(r + gamma*max(qtable[nextstate][y],qtable[nextstate][n]) - \
           qtable[state][action])
    
    if action == y and state == y:
        yy += 1
    elif action == y and state == n:
        yn += 1
    elif action == n and state == y:
        ny += 1
    else:
        nn +=1
        
    if(tempyn - tempny != yn - ny):
        temp1 = 1
        
    tempyn = yn
    tempny = ny
    
    if temp1 == 1:
        if yn>ny:
            w = w - (lamda*(yn-ny))
        elif yn<ny:
            w = w + (lamda*(ny-yn))
        temp1 = 0
        
print("yy = ",yy," ny = ",ny," yn = ",yn," nn = ",nn)

yy =  8  ny =  4  yn =  5  nn =  12


# Independent

In [9]:
for i in range(test_num):
    
    state = nextstate
    nextstate = opponent_Move()
    
    if qtable[state][y] > qtable[state][n]:
        action = y
    elif qtable[state][n] > qtable[state][y]:
        action = n
    else:
        action = rd.randint(0,1)
    
    r = reward(state,action)
    
    qtable[state][action] = qtable[state][action] + \
    alpha*(r + gamma*max(qtable[nextstate][y],qtable[nextstate][n]) - \
           qtable[state][action])
    
    if action == y and state == y:
        yy += 1
    elif action == y and state == n:
        yn += 1
    elif action == n and state == y:
        ny += 1
    else:
        nn +=1
        
    if(tempyn - tempny != yn - ny):
        temp1 = 1
        
    tempyn = yn
    tempny = ny
    
    if temp1 == 1:
        if yn>ny:
            w = w - (lamda*(yn-ny))
        elif yn<ny:
            w = w + (lamda*(ny-yn))
        temp1 = 0
        
print("yy = ",yy," ny = ",ny," yn = ",yn," nn = ",nn)

yy =  8  ny =  52  yn =  54  nn =  15
